# 3.3 线性回归的简单实现

## 生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [4]:
next(iter(data_iter))

[tensor([[-0.7354, -0.0735],
         [ 0.9370,  0.6088],
         [-0.4720, -0.6383],
         [-0.1247,  0.1136],
         [-1.3714,  0.4567],
         [ 0.5984,  0.1769],
         [-0.2959,  2.2876],
         [-0.6886, -0.0124],
         [-0.1628, -1.9516],
         [ 0.8217, -0.0532]]),
 tensor([[ 2.9662],
         [ 4.0093],
         [ 5.4119],
         [ 3.5805],
         [-0.0971],
         [ 4.7873],
         [-4.1715],
         [ 2.8673],
         [10.4988],
         [ 6.0148]])]

In [5]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [6]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [7]:
loss = nn.MSELoss()

In [8]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [9]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000300
epoch 2, loss 0.000101
epoch 3, loss 0.000101


In [10]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0005, 0.0006])
b的估计误差： tensor([0.0002])


## 练习题

练习题 1：如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率

学习率 / batch_size

练习题 2：查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失，即

$$
\begin{split}l(y,y') = \begin{cases}|y-y'| -\frac{\sigma}{2} & \text{ if } |y-y'| > \sigma \\ \frac{1}{2 \sigma} (y-y')^2 & \text{ 其它情况}\end{cases}\end{split}
$$

In [11]:
# huber损失对应Pytorch的SmoothL1损失
loss = nn.SmoothL1Loss(beta=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        #开始计算梯度
        trainer.zero_grad()
        l.backward()
        trainer.step()  #对所有参数进行更新
    print("epoch: {}, loss:{}".format(epoch + 1, l))

epoch: 1, loss:7.968719728523865e-05
epoch: 2, loss:9.276966738980263e-05
epoch: 3, loss:6.330489850370213e-05


练习题 3：如何访问线性回归的梯度

In [12]:
print(net[0].weight.grad)
print(net[0].bias.grad)

tensor([[0.0017, 0.0026]])
tensor([0.0035])
